In [1]:
# -*- coding: utf-8 -*-

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader


from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

In [2]:
import os

In [3]:
list_path_dir = os.listdir('./')
list_path_dir = [x for x in list_path_dir if x.endswith('.docx')]
list_path_dir = ['./' + x[:-5] for x in list_path_dir]
list_path_dir


['./Chương trình đào tạo ngành An toàn thông tin',
 './Chương trình đào tạo ngành Công nghệ Internet vạn vật – IoT',
 './Chương trình đào tạo ngành Công nghệ kỹ thuật điệnđiện tử',
 './Chương trình đào tạo ngành Công nghệ thông tin',
 './Chương trình đào tạo ngành Công nghệ tài chính (Fintech)',
 './Chương trình đào tạo ngành Công nghệ đa phương tiện',
 './Chương trình đào tạo ngành Khoa học máy tính (định hướng Khoa học dữ liệu)',
 './Chương trình đào tạo ngành Kế toán chuyên ngành Kế toán chuẩn quốc tế ACCA',
 './Chương trình đào tạo ngành Kế toán',
 './Chương trình đào tạo ngành Kỹ thuật điều khiển và tự động hoá',
 './Chương trình đào tạo ngành Kỹ thuật điện tử viễn thông',
 './Chương trình đào tạo ngành Marketing',
 './Chương trình đào tạo ngành Quản trị kinh doanh',
 './Chương trình đào tạo ngành Thương mại điện tử',
 './Chương trình đào tạo ngành Truyền thông đa phương tiện',
 './De an tuyen sinh 2024_Final',
 './Học viện Công nghệ Bưu chính Viễn thông công khai thông tin đào tạ

In [6]:
list_path_dir = [
#  './Chương trình đào tạo ngành An toàn thông tin',
#  './Chương trình đào tạo ngành Công nghệ Internet vạn vật – IoT',
#  './Chương trình đào tạo ngành Công nghệ kỹ thuật điệnđiện tử',
#  './Chương trình đào tạo ngành Công nghệ thông tin',
#  './Chương trình đào tạo ngành Công nghệ tài chính (Fintech)',
#  './Chương trình đào tạo ngành Công nghệ đa phương tiện',
#  './Chương trình đào tạo ngành Khoa học máy tính (định hướng Khoa học dữ liệu)',
#  './Chương trình đào tạo ngành Kế toán chuyên ngành Kế toán chuẩn quốc tế ACCA',
#  './Chương trình đào tạo ngành Kế toán',
#  './Chương trình đào tạo ngành Kỹ thuật điều khiển và tự động hoá',
#  './Chương trình đào tạo ngành Kỹ thuật điện tử viễn thông',
#  './Chương trình đào tạo ngành Marketing',
#  './Chương trình đào tạo ngành Quản trị kinh doanh',
#  './Chương trình đào tạo ngành Thương mại điện tử',
#  './Chương trình đào tạo ngành Truyền thông đa phương tiện',
#  './De an tuyen sinh 2024_Final',
#  './Phụ lục ĐH16 Chương trình đào tạo ngành Báo chí',
#  './Phụ lục ĐH17 Chương trình đào tạo ngành Công nghệ thông tin (chất lượng cao)',
#  './Phụ lục ĐH18 Chương trình đào tạo Cử nhân Công nghệ thông tin ( định hướng ứng dụng )',
#  './Phụ lục ĐH19 Chương trình giáo dục đại học Kỹ thuật dữ liệu (ngành Mạng máy tính và truyền thông dữ liệu)',
 './Ngành đào tạo',
 './test',
 './gen17052024',
 './gen21052024',
 "./Phòng đào tạo",
 './Điểm chuẩn PTIT năm 2023',
 './Thông tin tuyển sinh',
#  './Học phí',
 './Others'
 ]

In [13]:
from getpass import getpass
import os

# from langchain.llms import PromptLayerOpenAI

# PROMPTLAYER_API_KEY = getpass()
# os.environ["PROMPTLAYER_API_KEY"] = PROMPTLAYER_API_KEY

os.environ["OPENAI_API_KEY"] = "sk-proj-tZ5Uxrp7mtytZGbGRAMWT3BlbkFJIMLK05g3ZOlYdz8PIHkK"

In [21]:

# Khai bao bien
pdf_data_path = list_path_dir[0]
vector_db_path = "./vectorDB_05282024/"

def create_db_from_files():
    # Khai bao loader de quet toan bo thu muc dataa
    # loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls = PyPDFLoader)
    loader = DirectoryLoader(pdf_data_path, glob="*.txt", loader_cls = TextLoader)

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
    chunks = text_splitter.split_documents(documents)

    # Embeding
    embed_model = OpenAIEmbeddings()
    db = FAISS.from_documents(chunks, embed_model)
    db.save_local(vector_db_path)

    for i in list_path_dir[1:]:
        loader = DirectoryLoader(i, glob="*.txt", loader_cls = TextLoader)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
        chunks = text_splitter.split_documents(documents)
        db.add_documents(chunks)
        db.save_local(vector_db_path)
    
    return db


create_db_from_files()


In [4]:
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, model_kwargs={"top_p": 0.0001})

In [5]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.
Hãy trả lời thật chi tiết và chính xác nhất có thể.
Tất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt

Context: {context}
Question: {question}

"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.\nNếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.\nHãy trả lời thật chi tiết và chính xác nhất có thể.\nTất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt\n\nContext: {context}\nQuestion: {question}\n\n'))])

In [6]:
# Chain
chain = prompt | llm # hiểu là chain này sẽ gồm 2 phần, phần đầu tiên là prompt, phần thứ 2 là llm

In [7]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


embed_model = OpenAIEmbeddings()
vectorstore = FAISS.load_local(
    "./vectorDB_05172024/",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)


retriever = vectorstore.as_retriever(search_kwargs={"k": 5, "threshold": 0.5})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000166FF30DD10>, search_kwargs={'k': 5, 'threshold': 0.5})

In [28]:
# loader = DirectoryLoader("./Ngành đào tạo", glob="*.txt", loader_cls = TextLoader)

# documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=256)
# chunks = text_splitter.split_documents(documents)

# # Embeding
# embed_model = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(chunks, embed_model)

In [33]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 10, "threshold": 0.5})

# QA

In [33]:
question = "Thí sinh cần có Chứng chỉ tiếng Anh quốc tế TOEFL ITP từ bao nhiêu điểm trở lên để được xét tuyển theo phương thức 3?"
docs = retriever.get_relevant_documents(question)
docs

[Document(page_content='+) Thí sinh có Chứng chỉ tiếng Anh quốc tế trong thời hạn (tính đến ngày xét tuyển) đạt IELTS 5.5 trở lên hoặc TOEFL iBT 65 trở lên hoặc TOEFL ITP 513 trở lên; và có kết quả điểm trung bình chung học tập lớp 10, 11, 12 hoặc học kỳ 1 lớp 12 (nếu chưa có kết quả năm học lớp 12) đạt từ 7,5 trở lên và có hạnh kiểm Khá trở lên.\nd) Đối với Phương thức 4 - Xét tuyển dựa vào kết quả bài thi đánh giá năng lực (ĐGNL) hoặc đánh giá tư duy (ĐGTD): \nNgoài các yêu cầu theo quy định chung ở mục a) thì thí sinh cần có thêm một trong các điều kiện sau đây:\n+) Thí sinh có điểm thi đánh giá năng lực của Đại học quốc gia Hà Nội (HSA) năm 2024 từ 75 điểm trở lên;\n+) Thí sinh có điểm thi đánh giá năng lực của Đại học quốc gia Tp. Hồ Chí Minh (APT) năm 2024 từ 600 điểm trở lên;\n+) Thí sinh có điểm thi đánh giá tư duy của Đại học Bách khoa Hà Nội (TSA) năm 2024 từ 50 điểm trở lên.\n1.2 Thời gian nhận hồ sơ đăng ký xét tuyển:\na) Phương thức 1 – Xét tuyển tài năng\n(1) Xét tuyển th

In [34]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    HumanMessagePromptTemplate,
)
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
import re

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):  
    # docs.sort(key=lambda x: len(x.page_content), reverse=True)

    # for i in range(len(docs)):
    #     text = docs[i].page_content
    #     text = re.sub(r"\n", "|@#ript", text)
    #     text = re.sub(r"\s+", " ", text)
    #     text = re.sub(r"\|@#ript", "\n", text)
    #     text = re.sub(r" ,", "", text)

    #     docs[i].page_content = text

    # l = []
    # st = docs[0].page_content.strip()
    # for i in range(1, len(docs)):
    #     if docs[i].page_content.strip() not in st:
    #         l.append(docs[i])
    #         st += "\n"
    #         st += docs[i].page_content.strip()
    #         print(docs[i])
    # print(len(l))            
    # doc_strings = [format_document(doc, document_prompt) for doc in docs]
    # return document_separator.join(doc_strings)

    # from difflib import SequenceMatcher
    # # remove one of the pair of similar documents > 80% similarity
    
    # for i in range(len(docs)-1):
    #     for j in range(i+1, len(docs)):
    #         print(i, j)
    #         print(SequenceMatcher(None, docs[i].page_content, docs[j].page_content).ratio())
    #         if SequenceMatcher(None, docs[i].page_content, docs[j].page_content).ratio() > 0.9:
    #             docs[j].page_content = ""
    # [print(i) for i in docs]
    # docs = [i for i in docs if i.page_content != ""]       
    # print(len(docs))
    doc_strings = [format_document(doc, document_prompt) for doc in docs]     
    return document_separator.join(doc_strings)

In [35]:
docs_combined = _combine_documents(docs)
docs_combined

'+) Thí sinh có Chứng chỉ tiếng Anh quốc tế trong thời hạn (tính đến ngày xét tuyển) đạt IELTS 5.5 trở lên hoặc TOEFL iBT 65 trở lên hoặc TOEFL ITP 513 trở lên; và có kết quả điểm trung bình chung học tập lớp 10, 11, 12 hoặc học kỳ 1 lớp 12 (nếu chưa có kết quả năm học lớp 12) đạt từ 7,5 trở lên và có hạnh kiểm Khá trở lên.\nd) Đối với Phương thức 4 - Xét tuyển dựa vào kết quả bài thi đánh giá năng lực (ĐGNL) hoặc đánh giá tư duy (ĐGTD): \nNgoài các yêu cầu theo quy định chung ở mục a) thì thí sinh cần có thêm một trong các điều kiện sau đây:\n+) Thí sinh có điểm thi đánh giá năng lực của Đại học quốc gia Hà Nội (HSA) năm 2024 từ 75 điểm trở lên;\n+) Thí sinh có điểm thi đánh giá năng lực của Đại học quốc gia Tp. Hồ Chí Minh (APT) năm 2024 từ 600 điểm trở lên;\n+) Thí sinh có điểm thi đánh giá tư duy của Đại học Bách khoa Hà Nội (TSA) năm 2024 từ 50 điểm trở lên.\n1.2 Thời gian nhận hồ sơ đăng ký xét tuyển:\na) Phương thức 1 – Xét tuyển tài năng\n(1) Xét tuyển thẳng và ưu tiên xét tuyể

In [28]:
from difflib import SequenceMatcher
s = SequenceMatcher(None, "không logit", "không hợp lý")
print(s.ratio())

0.6086956521739131


In [29]:
import re
text = "4. CÁC CHÍNH SÁCH, HOẠT ĐỘNG HỖ TRỢ HỌC TẬP, SINH HOẠT CHO NGƯỜI HỌC  \n\nNgười học được cấp email riêng, cấp tài khoản để sử dụng phần mềm quản lý đào tạo trực tuyến để quản lý và theo dõi quá trình học tập của mình, tự tra cứu tiến độ kế hoạch học tập, kinh phí, học phí, kết quả học tập của cá nhân.  \n\nBên cạnh các hoạt động nằm trong chương trình đào tạo, sinh viên còn được đào tạo các kỹ năng mềm như kỹ năng thuyết trình,  kỹ năng làm việc nhóm, kỹ năng tạo lập văn bản tiếng việt. Đặc biệt, sinh viên còn được tham gia các hoạt động nghiên cứu khoa học, các câu lạc bộ do Học viện tổ chức.  \n\nChế độ, chính sách đối với sinh viên được thực hiện trên tinh thần công khai, công bằng  và đúng quy định. Tổ chức xét khen thưởng, kỷ luật được tiến hành thường xuyên và đúng quy chế."
text = re.sub(r"\n", "|@#ript", text)
text = re.sub(r"\s+", " ", text)
text = re.sub(r"\|@#ript", "\n", text)
text

'4. CÁC CHÍNH SÁCH, HOẠT ĐỘNG HỖ TRỢ HỌC TẬP, SINH HOẠT CHO NGƯỜI HỌC \n\nNgười học được cấp email riêng, cấp tài khoản để sử dụng phần mềm quản lý đào tạo trực tuyến để quản lý và theo dõi quá trình học tập của mình, tự tra cứu tiến độ kế hoạch học tập, kinh phí, học phí, kết quả học tập của cá nhân. \n\nBên cạnh các hoạt động nằm trong chương trình đào tạo, sinh viên còn được đào tạo các kỹ năng mềm như kỹ năng thuyết trình, kỹ năng làm việc nhóm, kỹ năng tạo lập văn bản tiếng việt. Đặc biệt, sinh viên còn được tham gia các hoạt động nghiên cứu khoa học, các câu lạc bộ do Học viện tổ chức. \n\nChế độ, chính sách đối với sinh viên được thực hiện trên tinh thần công khai, công bằng và đúng quy định. Tổ chức xét khen thưởng, kỷ luật được tiến hành thường xuyên và đúng quy chế.'

In [36]:
# Run
res = chain.invoke({"context":docs_combined,"question":question})
for r in res.content.split("\n"):
    print(r)

Thí sinh cần có Chứng chỉ tiếng Anh quốc tế TOEFL ITP từ 513 điểm trở lên để được xét tuyển theo phương thức 3.


# chat

In [40]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)

In [42]:
# Initialize OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

""" # Initialize Chroma
chroma = Chroma(embeddings)

# Initialize AzureChatOpenAI
azure_chat = AzureChatOpenAI() """

# Define your custom prompt template
prompt_template = "Dựa vào thông tin sau để trả lời câu hỏi của người dùng.\nNếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.\nHãy trả lời thật chi tiết và chính xác nhất có thể.\nTất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt\n\nContext: {context}\nQuestion: {question}\n"

# Create a ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate(prompt_template)

TypeError: Serializable.__init__() takes 1 positional argument but 2 were given

In [37]:
chain = RunnableParallel({
    "context": vectorstore.as_retriever(),
    "question": RunnablePassthrough()
}).prompt(chat_prompt_template).model(llm).parse(StrOutputParser()).with_types({
    "context": "str",
    "question": "str"
})

NameError: name 'RunnableParallel' is not defined

In [ ]:
from flask import Flask, session, request

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    username = request.json['username']
    question = request.json['question']

    # Retrieve chat history from session
    chat_history = session.get(username, [])

    # Add new question to chat history
    chat_history.append(question)

    # Run the chain
    answer = chain.run({"context": "Your Chroma context", "question": question})

    # Add answer to chat history
    chat_history.append(answer)

    # Store chat history in session
    session[username] = chat_history

    return {"answer": answer}

@app.route('/clear_chat_history', methods=['POST'])
def clear_chat_history():
    username = request.json['username']

    # Clear chat history for the given username
    session.pop(username, None)

    return {"status": "Chat history cleared"}

In [10]:
import os
from operator import itemgetter
from typing import List, Tuple

from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

### Ingest code - you may need to run this the first time
# # Load
# from langchain_community.document_loaders import WebBaseLoader
# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# data = loader.load()

# # Split
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

# # Add to vectorDB
# vectorstore = PineconeVectorStore.from_documents(
#     documents=all_splits, embedding=OpenAIEmbeddings(), index_name=PINECONE_INDEX_NAME
# )
# retriever = vectorstore.as_retriever()

# vectorstore = vectorstore
# retriever = vectorstore.as_retriever()

# Condense a chat history and follow-up question into a standalone question
_template = """Đưa ra cuộc trò chuyện sau đây và một câu hỏi tiếp theo, hãy diễn đạt lại câu hỏi tiếp theo thành một câu hỏi độc lập, bằng ngôn ngữ gốc của nó.
Cuộc trò chuyện: {chat_history}
Câu hỏi tiếp theo: {question}
Câu hỏi độc lập:"""  # noqa: E501

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# RAG answer synthesis prompt
template = """Trả lời câu hỏi của người dùng dựa trên thông tin sau đây.:
<context>
{context}
</context>"""
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

# Conversational Retrieval Chain
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


# User input
class ChatHistory(BaseModel):
    chat_history: List[Tuple[str, str]] = Field(..., extra={"widget": {"type": "chat"}})
    question: str


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(itemgetter("question")),
)

_inputs = RunnableParallel(
    {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "context": _search_query | retriever | _combine_documents,
    }
).with_types(input_type=ChatHistory)

chain = _inputs | ANSWER_PROMPT | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) | StrOutputParser()

c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
chat_history = []

In [ ]:
while True:
    question = input("Enter your question: ")
    chat_history.append((question, chain.invoke({"chat_history": chat_history, "question": question})))
    print(chat_history[-1][1])

Đẩy data
gồm 30 văn bản ba công khai
đạt: 0-14
hiếu: 15-29
dũng: lấy data csv thử trước

Tích hợp


In [8]:
import pandas as pd

df_val = pd.read_excel("./gen21052024/result_vi_test.xlsx")
df_val

,Intent,Question,Answer,Yes/No/Other,Lưu ý,Answer_AI
0,Chính sách học bổng,Học viện Công nghệ Bưu chính Viễn thông cấp ba...,NaN,NaN,NaN,NaN
1,Chính sách học bổng,Đối tượng xét cấp học bổng đặc biệt là ai?,NaN,NaN,NaN,NaN
2,Chính sách học bổng,Điều kiện để được nhận học bổng đặc biệt là gì?,NaN,NaN,NaN,NaN
3,Chính sách học bổng,Học viện Công nghệ Bưu chính Viễn thông cấp tố...,NaN,NaN,NaN,NaN
4,Chính sách học bổng,Ai được nhận học bổng toàn phần?,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
115,Điểm_cộng_đối_tượng_ưu_tiên,Đối tượng nào có thể được cộng cả 02 điểm và 0...,NaN,NaN,NaN,NaN
116,Điểm_cộng_đối_tượng_ưu_tiên,Tại sao quy định cộng điểm ưu tiên không áp dụ...,NaN,NaN,NaN,NaN
117,Điểm_cộng_đối_tượng_ưu_tiên,Liệt kê các lý do mà một đối tượng có thể được...,NaN,NaN,NaN,NaN
118,Điểm_cộng_đối_tượng_ưu_tiên,Tại sao việc cộng điểm ưu tiên giữa các nhóm đ...,NaN,NaN,NaN,NaN


In [31]:
df_val.iloc[63, 0]

'Thí sinh cần có Chứng chỉ tiếng Anh quốc tế TOEFL ITP từ bao nhiêu điểm trở lên để được xét tuyển theo phương thức 3?'

In [46]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("pairwise_string")

In [ ]:
i = 80
print("\n".join(df_val.iloc[i, 0].split(";")))
print("===================================")
print("\n".join(df_val.iloc[i, 1].split(";")))
print("===================================")
print("\n".join(df_val.iloc[i, 2].split(".")))
print("===================================")

In [1]:
# pip install openai

In [10]:
AIMessage = []
for i in range(len(df_val)):
    question = df_val['Question'][i]
    print(question)
    docs = retriever.get_relevant_documents(question)
    # print(docs)
    try:
        res = chain.invoke({"context":docs,"question":question})
        print(res.content)
        AIMessage.append(res.content)
    except Exception as e:
        print("Error", e)
        AIMessage.append("Error")

Học viện Công nghệ Bưu chính Viễn thông cấp bao nhiêu suất học bổng đặc biệt vào năm 2024?
Học viện Công nghệ Bưu chính Viễn thông cấp tối đa 30 suất học bổng đặc biệt vào năm 2024.
Đối tượng xét cấp học bổng đặc biệt là ai?
Đối tượng xét cấp học bổng đặc biệt là các thí sinh tham dự kỳ thi chọn đội tuyển quốc gia dự thi Olympic quốc tế, thí sinh đoạt giải Nhất, Nhì, Ba trong kỳ thi chọn học sinh giỏi quốc gia các môn Toán, Lý và Tin học; thí sinh có điểm thi tốt nghiệp THPT từ 29,0 điểm trở lên.
Điều kiện để được nhận học bổng đặc biệt là gì?
Để được nhận học bổng đặc biệt tại Học viện Công nghệ Bưu chính Viễn thông, thí sinh cần đáp ứng các điều kiện sau:
1. Tham dự kỳ thi chọn đội tuyển quốc gia dự thi Olympic quốc tế hoặc đoạt giải Nhất, Nhì, Ba trong kỳ thi chọn học sinh giỏi quốc gia các môn Toán, Lý và Tin học.
2. Đạt điểm thi tốt nghiệp THPT từ 29,0 điểm trở lên.
3. Đảm bảo điểm trung bình chung tích lũy năm học liên tục đạt từ loại Giỏi trở lên trong thời gian học tập.
Học việ

In [ ]:
docs = retriever.get_relevant_documents(question)
print(docs)
try:
    res = chain.invoke({"context":docs,"question":question})
    print(res.content)
    AIMessage.append(res.content)
except Exception as e:
    print("Error", e)
    AIMessage.append("Error")

In [11]:
df_val['Answer_AI'] = AIMessage
df_val.to_excel("./gen21052024/data_test_human_ai.xlsx", index=False)

In [13]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# Cau hinh
vector_db_path = "./vectordbSCE"


# Tao prompt template
def creat_prompt(template):
    prompt = PromptTemplate(template = template, input_variables=["context", "question"])
    return prompt


# Tao simple chain
def create_qa_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        retriever = db.as_retriever(search_kwargs = {"k":3}, max_tokens_limit=1024),
        return_source_documents = True,
        chain_type_kwargs= {'prompt': prompt}

    )
    return llm_chain

# Read tu VectorDB
def read_vectors_db():
    # Embeding
    # embed_model_id = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
    embed_model_id = "./exp_finetune/ft_s1"

    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

    embed_model = HuggingFaceEmbeddings(
        model_name=embed_model_id,
        model_kwargs={'device': device},
        encode_kwargs={'device': device, 'batch_size': 32}
    )
    db = FAISS.load_local(vector_db_path, embed_model)
    return db


# Bat dau thu nghiem
db = read_vectors_db()


# template = """<|im_start|>system
# Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời
# {context}
# <|im_end|>
# <|im_start|>user
# {question}<|im_end|>
# <|im_start|>assistant"""

# template = """<s>SYSTEM
# Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời
# {context}
# </s>
# <s>USER
# {question}</s>
# <s>ASSISTANT"""

template = """Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.
Tất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt

Context: {context}
Question: {question}

"""

prompt = creat_prompt(template)

llm_chain  =create_qa_chain(prompt, llm, db)

# Chay cai chain
# question = "Thủ tướng Phạm Văn Đồng đến thăm đơn vị và tham quan dàn máy tính điện tử Minsk lần đầu là  khi nào? "


c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
question = "Trụ sở chính học viện công nghệ bưu chính viễn thông ở đâu?"
# llm_chain.invoke({"query": question})
response = llm_chain.invoke({"query": question})
# print(response)

In [8]:
response

{'query': 'Điều kiện dự thi thạc sĩ?',
 'result': '\nĐể dự thi trình độ thạc sĩ tại Học viện, người dự thi phải có các điều kiện sau đây :\n\n1. Có bằng tốt nghiệp đại học hoặc tương đương trở lên.\n2. Có chứng chỉ ngoại ngữ (nếu cần thiết).\n3. Có kinh nghiệm làm việc phù hợp với ngành đào tạo được lựa chọn.\n4. Đáp ứng yêu cầu về năng lực chuyên môn, nghiệp vụ theo ngành đào tạo.\n5. Có đề án nghiên cứu hoặc lý lịch cá nhân thể hiện trình độ chuyên môn và năng lực của người dự thi trong lĩnh vực đào tạo đã chọn.\n6. Không có các hình thức kỷ luật buộc thôi việc theo quy định tại thời điểm xét tuyển.\n7. Đáp ứng điều kiện về sức khỏe để thực hiện chương trình đào tạo.\n8. Có cam kết học tập, nghiên cứu và tham gia các hoạt động của Học viện.\n<|im_end|>',
 'source_documents': [Document(page_content='Câu hỏi: Điều kiện tuyển sinh cho người dự thi tuyển sinh trình độ thạc sĩ tại Học viện là gì?\n\n- Bên cạnh các hoạt động nằm trong chương trình đào tạo, học viên cao học và NCS còn được 

In [9]:
for i in response['result'].split('\n'):
    print(i)


Để dự thi trình độ thạc sĩ tại Học viện, người dự thi phải có các điều kiện sau đây :

1. Có bằng tốt nghiệp đại học hoặc tương đương trở lên.
2. Có chứng chỉ ngoại ngữ (nếu cần thiết).
3. Có kinh nghiệm làm việc phù hợp với ngành đào tạo được lựa chọn.
4. Đáp ứng yêu cầu về năng lực chuyên môn, nghiệp vụ theo ngành đào tạo.
5. Có đề án nghiên cứu hoặc lý lịch cá nhân thể hiện trình độ chuyên môn và năng lực của người dự thi trong lĩnh vực đào tạo đã chọn.
6. Không có các hình thức kỷ luật buộc thôi việc theo quy định tại thời điểm xét tuyển.
7. Đáp ứng điều kiện về sức khỏe để thực hiện chương trình đào tạo.
8. Có cam kết học tập, nghiên cứu và tham gia các hoạt động của Học viện.
<|im_end|>


parent document
pdf to section
rag vietnamese
generate instruction following từ tệp vb của mình

tuning sbert với dữ liệu học viện
1 tập q-a

In [9]:
# Read tu VectorDB
def read_vectors_db():
    # Embeding
    embed_model_id = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"

    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

    embed_model = HuggingFaceEmbeddings(
        model_name=embed_model_id,
        model_kwargs={'device': device},
        encode_kwargs={'device': device, 'batch_size': 32}
    )
    db = FAISS.load_local(vector_db_path, embed_model)
    return db


# Bat dau thu nghiem
db = read_vectors_db()

No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.


In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

def load_corpus(directory, verbose=False):
    if verbose:
        print(f"Loading files in {directory}")

    reader = SimpleDirectoryReader(directory)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_data = load_corpus('./data/test/', verbose=True)

Loading files in ./data/test/
Loaded 38 docs


Parsing nodes: 100%|██████████| 38/38 [00:00<00:00, 118.86it/s]

Parsed 121 nodes


In [3]:
# pip install docx2txt

  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3973 sha256=70ad36615a36cc6d2046ab0a2f467ecd307d194fe55e49bb21b5e4d92f6669a6
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\0f\0e\7a\3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt
Note: you may need to restart the kernel to use updated packages.


In [6]:
train_data

[TextNode(id_='462bd185-a03c-4986-8f02-ce7e2b1e01e6', embedding=None, metadata={'file_name': 'e:/Learn-AI/Learn_DeepAI/llm/data/test/1.1._đối_tượng,_điều_kiện_tuyển_sinh.docx', 'file_path': 'e:\\Learn-AI\\Learn_DeepAI\\llm\\data\\test\\1.1._đối_tượng,_điều_kiện_tuyển_sinh.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 37834, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-06', 'last_accessed_date': None}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b6325ba5-143a-41e9-bcca-42b313267b2f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'e:/Learn-AI/Learn_DeepAI/llm/data/test/1.1._đối_tượng,_điều_kiện_tuyển_sinh.docx', 

In [8]:
for i in train_data:
    print(i.text)

1.1. Đối tượng, điều kiện tuyển sinh

a) Quy định chung: 

Đối tượng dự tuyển được xác định tại thời điểm xét tuyển (trước khi công bố kết quả xét tuyển chính thức là các thí sinh đã được công nhận tốt nghiệp trung học phổ thông (THPT) của Việt Nam hoặc có bằng tốt nghiệp của nước ngoài được công nhận trình độ tương đương hoặc đã có bằng tốt nghiệp trung cấp ngành nghề thuộc cùng nhóm ngành dự tuyển và đã hoàn thành đủ yêu cầu khối lượng kiến thức văn hóa cấp THPT theo quy định của pháp luật;

Có đủ thông tin cá nhân, hồ sơ dự tuyển theo quy định;

Đáp ứng các điều kiện khác theo quy chế tuyển sinh hiện hành của Bộ Giáo dục và Đào tạo và của Học viện.

b) Đối với phương thức xét tuyển dựa vào kết quả thi THPT năm 2023:

Ngoài các yêu cầu theo quy định chung ở mục a) thì thí sinh phải tham dự kỳ thi tốt nghiệp THPT năm 2023 với các bài thi/môn thi theo tổ hợp xét tuyển tương ứng các ngành của Học viện.

c) Đối với phương thức xét tuyển kết hợp thì ngoài các yêu cầu theo quy định chung ở

In [52]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions  # type: ignore
from langchain.output_parsers import ResponseSchema
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser


# Schema for structured response
response_schemas = [
    ResponseSchema(name="items", description="Loại mặt hàng,hàng hoá"),
    ResponseSchema(name="location", description="Địa điểm kinh doanh"),
    ResponseSchema(name="experimental", description="Năm kinh nghiệm buôn bán"),
    ResponseSchema(
        name="payment_method",
        description="Phương thức thanh toán tiền mặt hay trích công nợ",
    ),
    ResponseSchema(name="profit_margin", description="Tỉ suất lợi nhuận theo %"),
    ResponseSchema(name="profit", description="Lợi nhuận"),
    ResponseSchema(name="revenue", description="Doanh thu"),
    ResponseSchema(
        name="seller_method",
        description="Phương thức buôn bán: bán lẻ, bán buôn, bán sỉ, buôn chuyến...",
    ),
    ResponseSchema(
        name="repository",
        description="Hàng tồn kho, kho hàng, kho lạnh, tồn kho, lưu kho",
    ),
]


from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)


docs = """ 
1. Tình trạng pháp lý kinh doanh:
- Hộ kinh doanh: Cửa hàng Tâm Tiến
- Cửa hàng kinh doanh: Kiốt số A15, khách hàng thuê của hợp tác xã Vạn Lợi theo hợp đồng thuê ki ốt số 02/VL ký ngày 01/10/2008.
- Vị trí: Ki ốt A15 nằm tại khu Lò Ông Phước là trục đường chính di chuyển vào chợ Nành –Ninh Hiệp – 
Làng nghề buôn bán kinh doanh các mặt hàng vải vóc, may mặc quy mô lớn và lâu đời nhất miền Bắc. Ki ốt mặt đường rộng gần 10m, 
có xe bus lưu thông phục vu khách tỉnh và các phương tiện vận tải lưu thông.
- Kho hàng: KH có 02 kho hàng, nằm liền sau ki ốt A15 và C12 khu HTX Vạn Lợi, 
tổng diện tích 2 kho khoảng 300m2, đường vào kho rộng 7m, thuận tiện cho việc vận chuyển, bốc dỡ hàng hóa
- Nhân sự: 7 người.
+ 2 vợ chồng KH và con dâu trực tiếp quản lý, điều hành hoạt động kinh doanh.
+ 2 nhân viên bán hàng, bốc xếp vải tại Kiot.
+ 2 nhân viêc vận chuyển, bốc xếp hàng tại kho.
2. Kinh nghiệm quản lý: bà Nguyễn Thị Tâm sinh ra và lớn lên ở Ninh Hiệp và có kinh nghiệm hơn 
20 năm buôn bán mặt hàng vải các loại nên không chỉ sở hữu nhiều Ki ốt, (A15, C12, TTTM Sơn Long…), 
mà khách hàng còn có những đối tác đầu ra tiêu thụ khối lượng hàng hóa lớn và ổn định.
3. Phương thức kinh doanh:
- Nhập hàng (đầu vào):
+ Hàng hóa nhập chủ yếu từ Sài Gòn chiếm tỷ trọng 90%. KH thường xuyên bay vào Sài Gòn để lựa chọn mẫu mã, 
một số mặt hàng đặc thù trong nước không có, KH đặt xưởng tận gốc từ bên Trung Quốc hoặc gọi đầu mối phiên dịch đặt hàng hộ. 
KH có thể đến trực tiếp xưởng hoặc đặt hàng qua mạng sau đó bên đối tác sẽ vận chuyển hàng về tận kho tại HTX Vạn Lợi.
+ Hình thức thanh toán: Tiền mặt tại điểm giao nhận hàng.
+ Chi phí vận chuyển, đóng gói, nhân công, bốc vác liên quan đến nhập hàng đều do bên bán trả.
- Bán hàng (đầu ra):
+ Đối tác truyền thống: Chủ yếu là các xưởng may, các CSSX quần áo thời trang tại Ninh Hiệp, 
các cửa hàng bán lẻ tại địa bàn lân cận, một số các xưởng may lớn ở Hà Đông, La Phù, Thạch Thất…
+ Khách hàng vãng lãi:
Nhờ có vị trí đắc địa nên Khách hàng trên khắp các tỉnh thành đổ về chợ Ninh Hiệp đều di chuyển qua Ki ốt của KH, 
chỉ cần có nhu cầu là vào xem và lựa chọn mẫu mã. Thông thường đối tác trả tiền ngay (90%) hoặc trả chậm 
(chiếm khoảng 10% đối với khách quen).
- Hàng tồn kho: Đặc thù kinh doanh mặt hàng vải nên lượng hàng tồn kho trung bình tương đối lớn khoảng 13.420trđ, 
mùa cao điểm lên tới 15.000-16.000trđ
- Mùa vụ: Doanh thu bán hàng của KH tương đối đều, tuy nhiên mùa vụ cao điểm tập trung vào đầu thời điểm giao mùa: 
Xuân - Hạ; Thu - Đông (do các xưởng tập trung sản xuất ủ hàng cho vụ hè và vụ đông)
+ Tỷ lệ bán buôn 95%, bán lẻ 5%
4. Căn cứ sổ sách ghi chép tháng 10,11,12 do KH cung cấp 
- Doanh thu trung bình/tháng: 6.728trđ
- Chi phí hoạt động kinh doanh:
+ Giá vốn/tháng: 89%: 5.988trđ
+ Chi phí vận chuyển: 30trđ/tháng
+ Chi phí nhân công: 28trđ/tháng
+ Chi phí thuê cửa hàng: 0.52trđ/tháng
+ Chi phí khác: 30trđ/tháng
+ Lợi nhuận trung bình/tháng: 9.68%: 651trđ
=> Nguồn thu từ hoạt động sản xuất kinh doanh của KH ổn định
 """

import json

# Prompt template

prompt = PromptTemplate(
    template="""Bạn là chuyên gia trong lĩnh vực kinh doanh. Hãy trích xuất thông tin theo từng thuộc tính trong đoạn văn bản sau. Hãy đưa ra thông tin chính xác nhất. Nếu không có thông tin hãy trả về 'không có thông tin'. 
    Văn bản: {text}
    {format_instructions}""",
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)

# # Chain
chain = prompt | llm
respose = chain.invoke({"text": docs})
for i in respose.content.split("\n"):
    print(i)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"items": string  // Loại mặt hàng,hàng hoá
	"location": string  // Địa điểm kinh doanh
	"experimental": string  // Năm kinh nghiệm buôn bán
	"payment_method": string  // Phương thức thanh toán tiền mặt hay trích công nợ
	"profit_margin": string  // Tỉ suất lợi nhuận theo %
	"profit": string  // Lợi nhuận
	"revenue": string  // Doanh thu
	"seller_method": string  // Phương thức buôn bán: bán lẻ, bán buôn, bán sỉ, buôn chuyến...
	"repository": string  // Hàng tồn kho, kho hàng, kho lạnh, tồn kho, lưu kho
}
```
